In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from package.utils import DotDict, timer, get_config, load_data, merge_data

# from package.clustering import CONFIG_PATH

# conf = get_config(CONFIG_PATH)

from sklearn.preprocessing import StandardScaler

In [23]:
from package.clustering.get_model import get_model
from package.clustering.predict import predict_model, add_labels

# @timer
def train_model(model, X):
    return model.fit(X)

In [24]:
def run_test_data(raw_df, scaled_df):
    X = scaled_df.copy()

    label_df = None

    for k in range(2, 5):
        model_name = 'kmeans'
        model_param = {'n_clusters': k, 'random_state': 0}

        model = get_model(method=model_name, params=model_param)
        model = train_model(model=model, X=X)
        labels = predict_model(model=model, method=model_name, X=X)
        result = add_labels(X=X, labels=labels)
        result = result.rename(columns={'labels':f'k{k}'})
        result = result.set_index('party_rk')
        label_df = pd.concat(objs=[label_df, result], axis=1)

    main_df = pd.concat(objs=[raw_df, label_df], axis=1)
    main_df.index.name = 'party_rk'

    return main_df.reset_index()

In [25]:
landing_path = 's3://tli-crm-segmentation/data-test/load_dt=20250526'

groupbev_list = ['99pct', '1hold', '0anp', 'other']
groupfr_list = ['familyrider', 'familynorider', 'nofamilyrider', 'nofamilynorider']
group_list = []
for bev in groupbev_list:
    for fr in groupfr_list:
        group = f'{bev}_{fr}'
        group_list.append(group)

for group in group_list:
    path = f's3://tli-crm-segmentation/data-zones/neutral/grouped/feature_{group}.csv'
    raw_df = load_data(path=path).set_index('party_rk')

    path = f's3://tli-crm-segmentation/data-zones/neutral/grouped/feature_normalized_{group}.csv'
    scaled_df = load_data(path=path).set_index('party_rk')
    
    if 'norider' in group:
        col_list = [col for col in scaled_df.columns if '_health' not in col and '_ci' not in col]
        scaled_df = scaled_df[col_list]
    
    proxy = run_test_data(raw_df, scaled_df)
    
    file_name = f'EXPERIMENT_CRM_SEGMENTATION_{group.upper()}_KMEANS'
    proxy.to_csv(f'{landing_path}/{file_name}.csv', index=False)

function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/grouped/feature_99pct_familyrider.csv
Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/grouped/feature_99pct_familyrider.csv
function: load_data successfully executed at 0.1048743724822998s
function: load_data is starting...
loading data from: s3://tli-crm-segmentation/data-zones/neutral/grouped/feature_normalized_99pct_familyrider.csv
Succesfully loaded data from: s3://tli-crm-segmentation/data-zones/neutral/grouped/feature_normalized_99pct_familyrider.csv
function: load_data successfully executed at 0.08334684371948242s
function: get_model is starting...
function: get_model successfully executed at 3.62396240234375e-05s
function: predict_model is starting...
function: predict_model successfully executed at 0.0005660057067871094s
function: get_model is starting...
function: get_model successfully executed at 3.552436828613281e-05s
function: predict_model is star